# 集团成绩分析

清空数据库
``` python
delete = db.finalexam.delete_many({})
delete.deleted_count
```

In [1]:
import xlwings as xw
from pymongo import MongoClient, errors
from cytoolz import itertoolz

In [3]:
client = MongoClient('mongodb://10.57.0.92:27017')
db = client['maple_pds']
try:
    Cs = db.collection_names()
except errors.ServerSelectionTimeoutError:
    print('failed!')
else:
    print(db.collection_names())

['finalexam', 'users', 'examinfos', 'staffinfos', 'studentinfos', 'gradeinfos', 'courseinfos']


In [13]:
result = db.finalexam.aggregate([{'$group' : {
            '_id' : {"grade_id": "$grade_id", "score.math.state": "$score.math.state"},
            'avg' : {'$avg' : "$score.math.final"},
            'total': {'$sum' : 1}
        }
    }])

In [83]:
def score_staging(course):
    condition = [
        {"$match": {"score." + course + ".state": 0}},
        {'$group': {
            '_id': {"grade_id": "$grade_id"},
            'avg': {'$avg': "$score." + course + ".final"},
            'total': {'$sum': 1},
            's1': {'$sum': {'$cond': [{
                '$and': [
                    {"$lt": ["$score." + course + ".final", {
                        '$multiply': ["$score." + course + ".full", 0.6]}]},
                    {"$gte": ["$score." + course + ".final", {
                        '$multiply': ["$score." + course + ".full", 0]}]}
                ]
            }, 1, 0]}},
            's2': {'$sum': {'$cond': [{
                '$and': [
                    {"$lt": ["$score." + course + ".final", {
                            '$multiply': ["$score." + course + ".full", 0.8]}]},
                    {"$gte": ["$score." + course + ".final", {
                        '$multiply': ["$score." + course + ".full", 0.6]}]}
                ]
            }, 1, 0]}},
            's3': {'$sum': {'$cond': [{
                '$and': [
                    {"$lt": ["$score." + course + ".final", {
                            '$multiply': ["$score." + course + ".full", 0.9]}]},
                    {"$gte": ["$score." + course + ".final", {
                        '$multiply': ["$score." + course + ".full", 0.8]}]}
                ]
            }, 1, 0]}},
            's4': {'$sum': {'$cond': [{
                '$and': [
                    {"$lte": ["$score." + course + ".final", {
                            '$multiply': ["$score." + course + ".full", 1.0]}]},
                    {"$gte": ["$score." + course + ".final", {
                        '$multiply': ["$score." + course + ".full", 0.9]}]}
                ]
            }, 1, 0]}},
        }
        },
        {"$sort": {"_id": 1}}]
#     result = db.finalexam.aggregate(condition)
    return db.finalexam.aggregate(condition)

In [42]:
result = db.finalexam.aggregate([
    {"$match": {"score.math.state": 0}},
    {'$group': {
        '_id': {"grade_id": "$grade_id"},
        'avg': {'$avg': "$score.math.final"},
        'total': {'$sum': 1},
        's1': {'$sum': {'$cond': [{
            '$and': [
                {"$lt": ['$score.math.final', {
                        '$multiply': ['$score.math.full', 0.6]}]},
                {"$gte": ['$score.math.final', {
                    '$multiply': ['$score.math.full', 0]}]}
            ]
        }, 1, 0]}},
        's2': {'$sum': {'$cond': [{
            '$and': [
                        {"$lt": ['$score.math.final', {
                            '$multiply': ['$score.math.full', 0.8]}]},
                        {"$gte": ['$score.math.final', {
                            '$multiply': ['$score.math.full', 0.6]}]}
                        ]
        }, 1, 0]}},
        's3': {'$sum': {'$cond': [{
            '$and': [
                        {"$lt": ['$score.math.final', {
                            '$multiply': ['$score.math.full', 0.9]}]},
                        {"$gte": ['$score.math.final', {
                            '$multiply': ['$score.math.full', 0.8]}]}
                        ]
        }, 1, 0]}},
        's4': {'$sum': {'$cond': [{
            '$and': [
                        {"$lte": ['$score.math.final', {
                            '$multiply': ['$score.math.full', 1.0]}]},
                        {"$gte": ['$score.math.final', {
                            '$multiply': ['$score.math.full', 0.9]}]}
                        ]
        }, 1, 0]}},
    }
    }])
list(result)

[{'_id': {'grade_id': 8},
  'avg': 68.42105263157895,
  's1': 11,
  's2': 5,
  's3': 3,
  's4': 0,
  'total': 19},
 {'_id': {'grade_id': 7},
  'avg': 64.91891891891892,
  's1': 22,
  's2': 12,
  's3': 2,
  's4': 1,
  'total': 37},
 {'_id': {'grade_id': 6},
  'avg': 82.35185185185185,
  's1': 2,
  's2': 2,
  's3': 14,
  's4': 9,
  'total': 27},
 {'_id': {'grade_id': 5},
  'avg': 81.43478260869566,
  's1': 1,
  's2': 6,
  's3': 10,
  's4': 6,
  'total': 23},
 {'_id': {'grade_id': 4},
  'avg': 84.62068965517241,
  's1': 0,
  's2': 7,
  's3': 13,
  's4': 9,
  'total': 29},
 {'_id': {'grade_id': 3},
  'avg': 82.28947368421052,
  's1': 0,
  's2': 14,
  's3': 12,
  's4': 12,
  'total': 38},
 {'_id': {'grade_id': 2},
  'avg': 88.4322033898305,
  's1': 2,
  's2': 4,
  's3': 13,
  's4': 40,
  'total': 59},
 {'_id': {'grade_id': 1},
  'avg': 93.0,
  's1': 2,
  's2': 2,
  's3': 8,
  's4': 58,
  'total': 70}]

In [84]:
score_staging('chinese')

In [47]:
result = db.finalexam.aggregate([
    {"$match": {"score.math.state": 0}},
    {'$group': {
        '_id': {"grade_id": "$grade_id"},
        'avg': {'$avg': "$score.math.final"},
        'total': {'$sum': 1},
        's1': {'$sum': {'$cond': [{
            '$and': [
                {"$lt": ['$score.math.final', {
                        '$multiply': ['$score.math.full', 0.6]}]},
                {"$gte": ['$score.math.final', {
                    '$multiply': ['$score.math.full', 0]}]}
            ]
        }, 1, 0]}},
        's2': {'$sum': {'$cond': [{
            '$and': [
                        {"$lt": ['$score.math.final', {
                            '$multiply': ['$score.math.full', 0.8]}]},
                        {"$gte": ['$score.math.final', {
                            '$multiply': ['$score.math.full', 0.6]}]}
                        ]
        }, 1, 0]}},
        's3': {'$sum': {'$cond': [{
            '$and': [
                        {"$lt": ['$score.math.final', {
                            '$multiply': ['$score.math.full', 0.9]}]},
                        {"$gte": ['$score.math.final', {
                            '$multiply': ['$score.math.full', 0.8]}]}
                        ]
        }, 1, 0]}},
        's4': {'$sum': {'$cond': [{
            '$and': [
                        {"$lte": ['$score.math.final', {
                            '$multiply': ['$score.math.full', 1.0]}]},
                        {"$gte": ['$score.math.final', {
                            '$multiply': ['$score.math.full', 0.9]}]}
                        ]
        }, 1, 0]}},
    }
    },
    {"$sort":{"_id":1}}])
score_staging = []
for row in result:
    bai = [row['s'+str(i+1)]/row['total'] for i in range(4)]
    score_staging.append(list(reversed(bai))+[row['avg']])
print(*score_staging,sep='\n')

[0.8285714285714286, 0.11428571428571428, 0.02857142857142857, 0.02857142857142857, 93.0]
[0.6779661016949152, 0.22033898305084745, 0.06779661016949153, 0.03389830508474576, 88.4322033898305]
[0.3157894736842105, 0.3157894736842105, 0.3684210526315789, 0.0, 82.28947368421052]
[0.3103448275862069, 0.4482758620689655, 0.2413793103448276, 0.0, 84.62068965517241]
[0.2608695652173913, 0.43478260869565216, 0.2608695652173913, 0.043478260869565216, 81.43478260869566]
[0.3333333333333333, 0.5185185185185185, 0.07407407407407407, 0.07407407407407407, 82.35185185185185]
[0.02702702702702703, 0.05405405405405406, 0.32432432432432434, 0.5945945945945946, 64.91891891891892]
[0.0, 0.15789473684210525, 0.2631578947368421, 0.5789473684210527, 68.42105263157895]


In [8]:
stagings = list(itertoolz.sliding_window(2, [0, 60, 80, 90,100+1]))
print(stagings)

[(0, 60), (60, 80), (80, 90), (90, 101)]


In [18]:
score_staging = []
for i in range(1,10):
    cur = db.finalexam.find({'score.math.state':1,'grade_id':i})
    grade = [cur.count()]
    for m, n in stagings:
        # m ≤ score < n
        if i>= 7:
            m, n = m*1.2, n*1.2
        rng = {'$gte': m, '$lt': n}
        cur = db.finalexam.find({'score.math.final':rng, 'score.math.state':0, 'grade_id':i},{'_id': 0})
        grade.append(cur.count())
#         print('%s-%s:\t%s'% (m,n,cur.count()))

    print(grade)
    score_staging.append(list(reversed(grade)))

[2, 2, 2, 8, 58]
[0, 2, 4, 13, 40]
[0, 0, 14, 12, 12]
[0, 0, 7, 13, 9]
[0, 1, 6, 10, 6]
[0, 2, 2, 14, 9]
[0, 22, 12, 2, 1]
[1, 11, 5, 3, 0]
[0, 0, 0, 0, 0]


In [109]:
def score_staging_class(course):
    condition = [
        {"$match": {"score." + course + ".state": 0}},
        {'$group': {
            '_id': {"class_id": "$class_id", "grade_id": "$grade_id"},
            'avg': {'$avg': "$score." + course + ".final"},
            'total': {'$sum': 1},
            's1': {'$sum': {'$cond': [{
                '$and': [
                    {"$lt": ["$score." + course + ".final", {
                        '$multiply': ["$score." + course + ".full", 0.6]}]},
                    {"$gte": ["$score." + course + ".final", {
                        '$multiply': ["$score." + course + ".full", 0]}]}
                ]
            }, 1, 0]}},
            's2': {'$sum': {'$cond': [{
                '$and': [
                    {"$lt": ["$score." + course + ".final", {
                            '$multiply': ["$score." + course + ".full", 0.8]}]},
                    {"$gte": ["$score." + course + ".final", {
                        '$multiply': ["$score." + course + ".full", 0.6]}]}
                ]
            }, 1, 0]}},
            's3': {'$sum': {'$cond': [{
                '$and': [
                    {"$lt": ["$score." + course + ".final", {
                            '$multiply': ["$score." + course + ".full", 0.9]}]},
                    {"$gte": ["$score." + course + ".final", {
                        '$multiply': ["$score." + course + ".full", 0.8]}]}
                ]
            }, 1, 0]}},
            's4': {'$sum': {'$cond': [{
                '$and': [
                    {"$lte": ["$score." + course + ".final", {
                            '$multiply': ["$score." + course + ".full", 1.0]}]},
                    {"$gte": ["$score." + course + ".final", {
                        '$multiply': ["$score." + course + ".full", 0.9]}]}
                ]
            }, 1, 0]}},
        }
        },
        {"$sort": {"_id.grade_id": 1, "_id.class_id": 1}}]
#     result = db.finalexam.aggregate(condition)
    return db.finalexam.aggregate(condition)

In [110]:
list(score_staging_class('math'))

[{'_id': {'class_id': '201501', 'grade_id': 1},
  'avg': 95.34782608695652,
  's1': 0,
  's2': 0,
  's3': 1,
  's4': 22,
  'total': 23},
 {'_id': {'class_id': '201502', 'grade_id': 1},
  'avg': 93.04166666666667,
  's1': 1,
  's2': 0,
  's3': 5,
  's4': 18,
  'total': 24},
 {'_id': {'class_id': '201503', 'grade_id': 1},
  'avg': 90.6086956521739,
  's1': 1,
  's2': 2,
  's3': 2,
  's4': 18,
  'total': 23},
 {'_id': {'class_id': '201401', 'grade_id': 2},
  'avg': 89.13636363636364,
  's1': 1,
  's2': 1,
  's3': 3,
  's4': 17,
  'total': 22},
 {'_id': {'class_id': '201402', 'grade_id': 2},
  'avg': 89.97222222222223,
  's1': 0,
  's2': 1,
  's3': 6,
  's4': 11,
  'total': 18},
 {'_id': {'class_id': '201403', 'grade_id': 2},
  'avg': 86.15789473684211,
  's1': 1,
  's2': 2,
  's3': 4,
  's4': 12,
  'total': 19},
 {'_id': {'class_id': '201301', 'grade_id': 3},
  'avg': 81.04761904761905,
  's1': 0,
  's2': 9,
  's3': 8,
  's4': 4,
  'total': 21},
 {'_id': {'class_id': '201302', 'grade_id':

In [12]:
book = xw.Book('data/质量分析.xlsx')
book.sheets

Sheets([<Sheet [质量分析.xlsx]教学管理>, <Sheet [质量分析.xlsx]班级管理>, <Sheet [质量分析.xlsx]校区-基本>, ...])

In [13]:
school_teacher = book.sheets['校区-教师']

In [61]:
score_staging('chinese')

TypeError: 'list' object is not callable

In [85]:
rows = list(score_staging('chinese'))

In [86]:
score_staging = []
for row in rows:
    bai = [row['s'+str(i+1)]/row['total'] for i in range(4)]
    score_staging.append(list(reversed(bai))+[row['avg']])
print(*score_staging,sep='\n')
school_teacher.range('F11').value = score_staging

[0.8571428571428571, 0.1, 0.04285714285714286, 0.0, 94.22857142857143]
[0.8983050847457628, 0.03389830508474576, 0.05084745762711865, 0.01694915254237288, 93.03389830508475]
[0.47368421052631576, 0.3157894736842105, 0.18421052631578946, 0.02631578947368421, 84.6842105263158]
[0.5517241379310345, 0.2413793103448276, 0.20689655172413793, 0.0, 87.25862068965517]
[0.21739130434782608, 0.5217391304347826, 0.21739130434782608, 0.043478260869565216, 83.30434782608695]
[0.5925925925925926, 0.25925925925925924, 0.14814814814814814, 0.0, 89.29629629629629]
[0.24324324324324326, 0.1891891891891892, 0.4864864864864865, 0.08108108108108109, 92.4054054054054]
[0.0, 0.0, 0.5789473684210527, 0.42105263157894735, 73.89473684210526]


In [52]:
cur = db.finalexam.find({'score.math.state':1,'grade_id':1})
cur.count()

2

In [58]:
score_staging

[[2, 2, 8, 51, 2],
 [2, 4, 13, 39, 0],
 [0, 14, 12, 11, 0],
 [0, 7, 13, 9, 0],
 [1, 6, 10, 6, 0],
 [2, 2, 14, 9, 0],
 [22, 12, 2, 1, 0],
 [11, 5, 3, 0, 1],
 [0, 0, 0, 0, 0]]

In [32]:
cur = db.finalexam.find({'grade_id':1},{'_id': 0})

In [15]:
cur.count()

72

In [35]:
def group_score(doc):
    for m, n in itertoolz.sliding_window(2, staging):
        # m ≤ score < n
        rng = {'$gte': m, '$lt': n}
        doc['score']['math']['final']
        cur = db.finalexam.find({'score.math.final':rng,'grade_id':1},{'_id': 0})
        print('%s-%s:\t%s'% (m,n,cur.count()))

In [28]:
cur = db.finalexam.find({'score.math.0':{'$gte': 80, '$lte': 89},'grade_id':1},{'_id': 0})

In [29]:
for row in cur:
    print(row)

{'exam_id': '2016201001', 'score': {'math': [85.5, 100], 'chinese': [93.0, 100]}, 'name': '李嘉耀', 'class_id': '201501', 'grade_id': 1}
{'exam_id': '2016201001', 'score': {'math': [87.5, 100], 'chinese': [82.0, 100]}, 'name': '王锦轩', 'class_id': '201502', 'grade_id': 1}
{'exam_id': '2016201001', 'score': {'math': [88.5, 100], 'chinese': [90.5, 100]}, 'name': '王馨悦', 'class_id': '201502', 'grade_id': 1}
{'exam_id': '2016201001', 'score': {'math': [82.0, 100], 'chinese': [91.5, 100]}, 'name': '赵航标', 'class_id': '201502', 'grade_id': 1}
{'exam_id': '2016201001', 'score': {'math': [87.5, 100], 'chinese': [84.0, 100]}, 'name': '吴金辉', 'class_id': '201503', 'grade_id': 1}
{'exam_id': '2016201001', 'score': {'math': [87.0, 100], 'chinese': [84.5, 100]}, 'name': '赵启志', 'class_id': '201503', 'grade_id': 1}
